## Mathjax custom 

$ \newcommand{\opexpect}[3]{\langle #1 \vert #2 \vert #3 \rangle} $
$ \newcommand{\rarrow}{\rightarrow} $
$ \newcommand{\bra}{\langle} $
$ \newcommand{\ket}{\rangle} $

$ \newcommand{\up}{\uparrow} $
$ \newcommand{\down}{\downarrow} $

$ \newcommand{\mb}[1]{\mathbf{#1}} $
$ \newcommand{\mc}[1]{\mathcal{#1}} $
$ \newcommand{\mbb}[1]{\mathbb{#1}} $
$ \newcommand{\mf}[1]{\mathfrak{#1}} $

$ \newcommand{\vect}[1]{\boldsymbol{\mathrm{#1}}} $
$ \newcommand{\expect}[1]{\langle #1\rangle} $

$ \newcommand{\innerp}[2]{\langle #1 \vert #2 \rangle} $
$ \newcommand{\fullbra}[1]{\langle #1 \vert} $
$ \newcommand{\fullket}[1]{\vert #1 \rangle} $
$ \newcommand{\supersc}[1]{^{\text{#1}}} $
$ \newcommand{\subsc}[1]{_{\text{#1}}} $
$ \newcommand{\sltwoc}{SL(2,\mathbb{C})} $
$ \newcommand{\sltwoz}{SL(2,\mathbb{Z})} $

$ \newcommand{\utilde}[1]{\underset{\sim}{#1}} $

## Quantum Error Correction

References: [ADH, 2014, Sec. 3](http://arxiv.org/abs/1411.7041) (& [CGL, 1999](http://arxiv.org/abs/quant-ph/9901025))

In [1]:
import matplotlib
matplotlib.use('qt4agg')
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

### Erasure Correction

Reference: ADH, 2014, Sec. 3.1

A 'qutrit' is a quantum state defined on a three-dimensional Hilbert space $\mc{H}_3$. A state $\fullket{\psi} \in \mc{H}_3$ can be written as:

$$ \fullket{\psi} = \sum_{i=0}^2 a_i \fullket{i} $$

where $ \fullket{i} \in \{\fullket{0}, \fullket{1}, \fullket{2}\}$

Alice wishes to send the state $\fullket{\psi}$. For this purpose she uses the three-qutrit state:

$$ \fullket{\tilde \psi} = \sum_{i=0}^2 a_i \fullket{\tilde i} $$

where:

\begin{align}
    \fullket{\tilde 0} & = \frac{1}{\sqrt{3}}\left( \fullket{000} + \fullket{111} + \fullket{222} \right) \\
    \fullket{\tilde 1} & = \frac{1}{\sqrt{3}}\left( \fullket{012} + \fullket{120} + \fullket{201} \right) \\
    \fullket{\tilde 2} & = \frac{1}{\sqrt{3}}\left( \fullket{021} + \fullket{102} + \fullket{210} \right)
\end{align}

In QuTiP, a single qutrit state can be created as follows:

In [3]:
q0 = basis(3,0)
q1 = basis(3,1)
q2 = basis(3,2)

In [4]:
q0, q1, q2

(Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
 Qobj data =
 [[ 1.]
  [ 0.]
  [ 0.]], Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
 Qobj data =
 [[ 0.]
  [ 1.]
  [ 0.]], Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
 Qobj data =
 [[ 0.]
  [ 0.]
  [ 1.]])

It will be more convenient to put these states into an numpy ndarray object

In [5]:
qutrit_basis = np.ndarray((3), dtype=Qobj)
qutrit_basis = [q0, q1, q2]

Create an alias with shorter name

In [6]:
qb = qutrit_basis

The three qutrit basis states can then be expressed as elements of a three-index array object

In [7]:
qutrit3_basis = np.ndarray((3,3,3), dtype=Qobj)
q3b = qutrit3_basis

In [8]:
for i in range(2):
    for j in range(2):
        for k in range(3):
            q3b[i][j][k] = tensor([qb[i],qb[j],qb[k]])

In [9]:
def ndit_basis(N=2,n=2):
    '''returns a numpy array of Qobj elements, each element of which is a basis state of the tensor product space
    of N ndits (n)'''
    return

We can now create the particular combinations of the 3-qutrit states which will correspond to the basis states Alice used to send her message

In [10]:
alice3_basis = np.ndarray(3, dtype=Qobj)
a3b = alice3_basis

In [11]:
a3b[0] = 1/(np.sqrt(3))*(q3b[0,0,0] + q3b[1,1,1] + q3b[2,2,2])
a3b[1] = 1/(np.sqrt(3))*(q3b[0,1,2] + q3b[1,2,0] + q3b[2,0,1])
a3b[2] = 1/(np.sqrt(3))*(q3b[0,2,1] + q3b[1,0,2] + q3b[2,1,0])

Construct state $\fullket{\tilde \psi}$ with random coefficients out of Alice's three-qutrit states.

In [13]:
a = np.ndarray((3))
dont_accept = True
while dont_accept:
    a[0] = np.random.random_sample()
    a[1] = np.random.random_sample()
    if (a[0]**2 + a[1]**2) < 1:
        dont_accept = False
a[2] = np.sqrt(1 - a[0]**2 - a[1]**2)
a

array([ 0.3220762 ,  0.34865907,  0.88017258])

In [14]:
a[0]**2 + a[1]**2 + a[2]**2

1.0

In [15]:
def random_normalized_coefs(N=3):
    '''Returns N floats, whose squared sum is 1'''
    a = np.ndarray((N))
    dont_accept = True
    sum = 0
    while dont_accept:
        for i in range(N-1):
            a[i] = np.random.random_sample()
            sum+= a[i]**2
        if sum < 1:
            dont_accept = False
    a[N-1] = np.sqrt(1 - sum)
    return a

In [16]:
np.linalg.norm(a)

1.0

In [17]:
a = random_normalized_coefs(len(a3b))
psi_3q = Qobj
for i in range(len(a3b)):
    psi_3q += a[i]*a3b[i]
psi_3q

/Users/deepak/anaconda/envs/python3/lib/python3.5/site-packages/qutip/qobj.py:301: UserWarning: Initializing Qobj from unsupported type: <class 'type'>
  builtins.type(inpt))


Quantum object: dims = [[3, 3, 3], [1, 1, 1]], shape = (27, 1), type = ket
Qobj data =
[[ 0.2135584 ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.41357974]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.34158153]
 [ 0.        ]
 [ 0.2135584 ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]

In [18]:
print(psi_3q.data)

  (0, 0)	(0.213558398256+0j)
  (5, 0)	(0.413579737501+0j)
  (11, 0)	(0.341581534332+0j)
  (13, 0)	(0.213558398256+0j)


In [21]:
ket2dm(psi_3q)

Quantum object: dims = [[3, 3, 3], [3, 3, 3]], shape = (27, 27), type = oper, isherm = True
Qobj data =
[[ 0.04560719  0.          0.          0.          0.          0.08832343
   0.          0.          0.          0.          0.          0.07294761
   0.          0.04560719  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.     

In [22]:
import qutip.visualization as qvis

In [23]:
qutip.visualization._sequence_to_index([1,0,0],[2,2,2])

4

In [24]:
qvis._sequence_to_latex([1,0,1])

'$\\left|101\\right\\rangle$'

In [25]:
qvis._index_to_sequence(27,[3,3,3])

[0, 0, 0]

In [26]:
print(psi_3q.data)

  (0, 0)	(0.213558398256+0j)
  (5, 0)	(0.413579737501+0j)
  (11, 0)	(0.341581534332+0j)
  (13, 0)	(0.213558398256+0j)


In [27]:
data = psi_3q.data

In [28]:
psi_3q.norm()

0.6155814509866206

In [29]:
data

<27x1 sparse matrix of type '<class 'numpy.complex128'>'
	with 4 stored elements in Compressed Sparse Row format>

In [30]:
def ket_to_latex(qket):
    '''Given a state in the form of a QuTiP ket as input (qket), returns the latex expression for the
    state in terms of a superposition of basis states for the underlying Hilbert space'''
    
    return

In [31]:
data.nonzero()

(array([ 0,  5, 11, 13], dtype=int32), array([0, 0, 0, 0], dtype=int32))

In [33]:
data.data

array([ 0.21355840+0.j,  0.41357974+0.j,  0.34158153+0.j,  0.21355840+0.j])

In [34]:
data.indptr

array([0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4], dtype=int32)

In [35]:
data.indices

array([0, 0, 0, 0], dtype=int32)

In [36]:
data.todense()

matrix([[ 0.21355840+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.41357974+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.34158153+0.j],
        [ 0.00000000+0.j],
        [ 0.21355840+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j],
        [ 0.00000000+0.j]])

In [37]:
for i in data:
    print(i)

  (0, 0)	(0.213558398256+0j)




  (0, 0)	(0.413579737501+0j)





  (0, 0)	(0.341581534332+0j)

  (0, 0)	(0.213558398256+0j)















In [38]:
data[5,0]

(0.41357973750078092+0j)

In [39]:
l = [0,5,11,13]
kets = []
for i in range(len(l)):
    print (l[i])
    kets.append(ket(qvis._index_to_sequence(l[i],[3,3,3]),dim=3))

0
5
11
13


In [40]:
kets;

In [41]:
state = Qobj()
for i in range(len(l)):
    state = state + data[l[i],0]*kets[i]

In [42]:
state

Quantum object: dims = [[3, 3, 3], [1, 1, 1]], shape = (27, 1), type = ket
Qobj data =
[[ 0.2135584 ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.41357974]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.34158153]
 [ 0.        ]
 [ 0.2135584 ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]

In [43]:
print(state.data)

  (0, 0)	(0.213558398256+0j)
  (5, 0)	(0.413579737501+0j)
  (11, 0)	(0.341581534332+0j)
  (13, 0)	(0.213558398256+0j)


In [44]:
psi_3q == state

True

In [46]:
ket(qvis._index_to_sequence(0,[3,3,3]),dim=3)

Quantum object: dims = [[3, 3, 3], [1, 1, 1]], shape = (27, 1), type = ket
Qobj data =
[[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

In [47]:
ket?